In [64]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
# afriberta = AutoModelForMaskedLM.from_pretrained("castorini/afriberta_large")
tokenizer_afriberta = AutoTokenizer.from_pretrained("castorini/afriberta_large")


# xlmr = AutoModelForMaskedLM.from_pretrained("nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large")
tokenizer_xlmr = AutoTokenizer.from_pretrained("nreimers/mMiniLMv2-L12-H384-distilled-from-XLMR-Large")

# xlmr.save_pretrained("../models/xlmr_distilled")
# afriberta.save_pretrained("../models/afriberta")

afriberta = AutoModelForMaskedLM.from_pretrained("../models/afriberta")
xlmr = AutoModelForMaskedLM.from_pretrained("../models/xlmr_distilled")

/home/opc/miniconda3/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [8]:
text = "مرحبا كيف حالك"

print(tokenizer_afriberta.tokenize(text))
print(tokenizer_xlmr.tokenize(text))

['▁', 'مرحبا', '▁', 'كيف', '▁', 'حا', 'ل', 'ك']
['▁م', 'رحب', 'ا', '▁كيف', '▁حال', 'ك']


In [8]:
model = xlmr
for name, module in model.named_modules():
    print(name, module)


 XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 384, padding_idx=1)
      (position_embeddings): Embedding(514, 384, padding_idx=1)
      (token_type_embeddings): Embedding(1, 384)
      (LayerNorm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=384, out_features=384, bias=True)
              (key): Linear(in_features=384, out_features=384, bias=True)
              (value): Linear(in_features=384, out_features=384, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=384, out_features=384, bias=True)
 

In [65]:
model = afriberta
for name, module in model.named_modules():
    print(name, module)

 XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(70006, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-9): 10 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
   

In [81]:
import torch
embedding = model.base_model.get_input_embeddings()
copy = embedding.weight.clone()

print(embedding.weight.shape)
print(type(embedding))
print(type(embedding.weight))
print(type(embedding.weight.data))
print(embedding.weight.data)


torch.Size([70006, 768])
<class 'torch.nn.modules.sparse.Embedding'>
<class 'torch.nn.parameter.Parameter'>
<class 'torch.Tensor'>
tensor([[ 0.0066, -0.0544, -0.0122,  ...,  0.0253, -0.0409,  0.0022],
        [ 0.0014, -0.0939, -0.0466,  ...,  0.0630, -0.1047, -0.0675],
        [ 0.0011, -0.0057,  0.0189,  ..., -0.0010,  0.0760,  0.0195],
        ...,
        [ 0.0105, -0.0434, -0.0208,  ...,  0.0209, -0.0526,  0.0214],
        [-0.0286,  0.0250, -0.0174,  ..., -0.0474,  0.0257, -0.0088],
        [ 0.0075, -0.0547, -0.0067,  ..., -0.0089,  0.0010, -0.0029]])


In [91]:
vocab_size = embedding.weight.shape[0]
permutation = torch.randperm(vocab_size)

embedding.weight.data = embedding.weight.data[permutation, :]
model.base_model.set_input_embeddings(embedding)
model.base_model.get_input_embeddings().weight.data

tensor([[-0.0192,  0.0456,  0.0017,  ..., -0.0313,  0.0531, -0.0389],
        [-0.0191,  0.0401, -0.0203,  ..., -0.0047,  0.1012, -0.0017],
        [-0.0140,  0.0728, -0.0112,  ..., -0.0900,  0.0288,  0.0364],
        ...,
        [ 0.0071, -0.0141, -0.0514,  ..., -0.0081, -0.0669, -0.0096],
        [ 0.0232, -0.0285, -0.0173,  ..., -0.0430,  0.0374, -0.0250],
        [-0.0344, -0.0044, -0.0261,  ..., -0.1030,  0.0151,  0.0507]])

In [96]:
model.save_pretrained("../models/afriberta_rand")

AttributeError: 'XLMRobertaTokenizerFast' object has no attribute 'save'

In [ ]:

for name, param in model.named_parameters():
    print(name, param.shape, type(param))

In [101]:
import os
current_value = os.getenv("PYTORCH_CUDA_ALLOC_CONF", "N/A")
print("Current max_split_size_mb:", current_value)

Current max_split_size_mb: N/A


In [93]:
afriberta_rand = AutoModelForMaskedLM.from_pretrained("../models/afriberta_rand")
afriberta_rand.base_model.get_input_embeddings().weight.data


tensor([[-0.0192,  0.0456,  0.0017,  ..., -0.0313,  0.0531, -0.0389],
        [-0.0191,  0.0401, -0.0203,  ..., -0.0047,  0.1012, -0.0017],
        [-0.0140,  0.0728, -0.0112,  ..., -0.0900,  0.0288,  0.0364],
        ...,
        [ 0.0071, -0.0141, -0.0514,  ..., -0.0081, -0.0669, -0.0096],
        [ 0.0232, -0.0285, -0.0173,  ..., -0.0430,  0.0374, -0.0250],
        [-0.0344, -0.0044, -0.0261,  ..., -0.1030,  0.0151,  0.0507]])